In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import pymongo
import time

In [2]:
#executable_path = {'executable_path': ChromeDriverManager().install()}
#browser = Browser('chrome', **executable_path, headless=False)
#url = "https://www.boxofficemojo.com/chart/top_lifetime_gross/" 
#browser.visit(url)

#time.sleep(1)
#html = browser.html
#soup = BeautifulSoup(html, "html.parser")

#for x in range(4):
#    movie_gross = soup.find_all("tr")
#    for movie in movie_gross:
#        try:
#            title = movie.find("td", class_ = "a-text-left mojo-field-type-title").get_text()
#            gross = movie.find("td", class_ = "a-text-right mojo-field-type-money").get_text()
#            year = movie.find("td", class_ = "a-text-left mojo-field-type-year").get_text()
#        except:
#           print("Not found")
#    #try:
#   browser.links.find_by_partial_text("Next page").click()
         
    #except:
        #print("Scraping Complete")
#browser.quit()

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/" 
browser.visit(url)

time.sleep(2)
html = browser.html
soup = BeautifulSoup(html, "html.parser")


titles = []
world_gross = []
domestic_gross = []
domestic_prct = []
foreign_gross = []
foreign_prct = []
years = []
for x in range(5):
    if(x != 0):
        browser.links.find_by_partial_text("Next page").click()
    movie_gross = soup.find_all("tr")
    for movie in movie_gross:
        try:
            titles.append(movie.find("td", class_ = "a-text-left mojo-field-type-title").get_text())
            
            gross = movie.find_all("td", class_ = "a-text-right mojo-field-type-money")
            world_gross.append(gross[0].text)
            domestic_gross.append(gross[1].text)
            foreign_gross.append(gross[2].text)
            
            prct = movie.find_all("td", class_ = "a-text-right mojo-field-type-percent")
            domestic_prct.append(prct[0].text)
            foreign_prct.append(prct[1].text)
            
            years.append(movie.find("td", class_ = "a-text-left mojo-field-type-year").get_text())
        except:
            print("Not found")
    #browser.links.find_by_partial_text("Next page").click()
browser.quit()

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.21M/6.21M [00:01<00:00, 4.58MB/s]


Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found
Not found


In [9]:
movies_dict = {}
movies_dict["Title"] = titles
movies_dict["Worldwide Lifetime Gross"] = world_gross
movies_dict["Domestic Lifetime Gross"] = domestic_gross
movies_dict["Domestic %"] = domestic_prct
movies_dict["Foreign Lifetime Gross"] = foreign_gross
movies_dict["Foreign %"] = foreign_prct
movies_dict["Year"] = years
movies_df = pd.DataFrame.from_dict(movies_dict)
movies_df

,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,Avatar,"$2,847,397,339","$760,507,625",26.7%,"$2,086,889,714",73.3%,2009
1,Avengers: Endgame,"$2,797,501,328","$858,373,000",30.7%,"$1,939,128,328",69.3%,2019
2,Titanic,"$2,201,647,264","$659,363,944",30%,"$1,542,283,320",70%,1997
3,Star Wars: Episode VII - The Force Awakens,"$2,069,521,700","$936,662,225",45.3%,"$1,132,859,475",54.7%,2015
4,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%,2018
...,...,...,...,...,...,...,...
995,Ted,"$549,368,315","$218,815,487",39.8%,"$330,552,828",60.2%,2012
996,American Sniper,"$547,459,020","$350,159,020",64%,"$197,300,000",36%,2014
997,Mission: Impossible II,"$546,388,108","$215,409,889",39.4%,"$330,978,219",60.6%,2000
998,Detective Chinatown 2,"$544,185,156","$1,983,984",0.4%,"$542,201,172",99.6%,2018


In [20]:
movies_df["Worldwide Lifetime Gross"] = movies_df["Worldwide Lifetime Gross"].replace("[$,]", "", regex=True).astype(float)
movies_df = movies_df[movies_df["Domestic Lifetime Gross"] != "-"]
movies_df["Domestic Lifetime Gross"] = movies_df["Domestic Lifetime Gross"].replace("[$,]", "", regex=True).astype(float)
movies_df["Foreign Lifetime Gross"] = movies_df["Foreign Lifetime Gross"].replace("[$,]", "", regex=True).astype(float)
movies_df

,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,Avatar,2.847397e+09,760507625.0,26.7%,2.086890e+09,73.3%,2009
1,Avengers: Endgame,2.797501e+09,858373000.0,30.7%,1.939128e+09,69.3%,2019
2,Titanic,2.201647e+09,659363944.0,30%,1.542283e+09,70%,1997
3,Star Wars: Episode VII - The Force Awakens,2.069522e+09,936662225.0,45.3%,1.132859e+09,54.7%,2015
4,Avengers: Infinity War,2.048360e+09,678815482.0,33.1%,1.369544e+09,66.9%,2018
...,...,...,...,...,...,...,...
995,Ted,5.493683e+08,218815487.0,39.8%,3.305528e+08,60.2%,2012
996,American Sniper,5.474590e+08,350159020.0,64%,1.973000e+08,36%,2014
997,Mission: Impossible II,5.463881e+08,215409889.0,39.4%,3.309782e+08,60.6%,2000
998,Detective Chinatown 2,5.441852e+08,1983984.0,0.4%,5.422012e+08,99.6%,2018


In [23]:
movies_df["Domestic %"] = movies_df["Domestic %"].replace("[%<]", "", regex=True).astype(float)
movies_df["Foreign %"] = movies_df["Foreign %"].replace("[%<]", "", regex=True).astype(float)
movies_df

C:\Users\cabon\AppData\Local\Temp/ipykernel_1384/3907320167.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df["Foreign %"] = movies_df["Foreign %"].replace("[%<]", "", regex=True).astype(float)


,Title,Worldwide Lifetime Gross,Domestic Lifetime Gross,Domestic %,Foreign Lifetime Gross,Foreign %,Year
0,Avatar,2.847397e+09,760507625.0,26.7,2.086890e+09,73.3,2009
1,Avengers: Endgame,2.797501e+09,858373000.0,30.7,1.939128e+09,69.3,2019
2,Titanic,2.201647e+09,659363944.0,30.0,1.542283e+09,70.0,1997
3,Star Wars: Episode VII - The Force Awakens,2.069522e+09,936662225.0,45.3,1.132859e+09,54.7,2015
4,Avengers: Infinity War,2.048360e+09,678815482.0,33.1,1.369544e+09,66.9,2018
...,...,...,...,...,...,...,...
995,Ted,5.493683e+08,218815487.0,39.8,3.305528e+08,60.2,2012
996,American Sniper,5.474590e+08,350159020.0,64.0,1.973000e+08,36.0,2014
997,Mission: Impossible II,5.463881e+08,215409889.0,39.4,3.309782e+08,60.6,2000
998,Detective Chinatown 2,5.441852e+08,1983984.0,0.4,5.422012e+08,99.6,2018


In [24]:
movies_df.to_csv("box_office_mojo_movies.csv", index=False)